In [ ]:
import json
import time
import importlib
from datetime import datetime, timedelta, timezone
import requests
import pandas as pd

# Probabilistic Forecast Handler Class

In [ ]:
### Create a handler class
class PFXHandler:
    pfx_url = "https://service.solaranywhere.com/api/v2/ProbabilisticForecast"
    def __init__(self, api_key=None):
        assert api_key is not None, "You need to set a SolarAnywhere API key with kwarg: api_key=[api_key]"
        self.api_key = api_key

    def create_headers(self):
        """Create the headers for each request"""
        headers = {
            'Accept': 'application/json',
            'Accept-Charset': 'utf-8',
            'Content-Type': 'application/json',
            'X-Api-Key': self.api_key
        }
        return headers

    def post_request(
        self,
        request_body:dict
    ):
        """Initiates the request for a prob forecast.
        Input:
            request_body:dict: The specifications for the prob forecast request.
                See: https://apidocs.solaranywhere.com/#59e35b6b-91b7-47cc-94a7-fffbbb6f2363
        Output:
            post_response(request response): The returned POST request response from the SolarAnywhere API.
        """
        headers = self.create_headers()
        post_response = requests.request("POST", self.pfx_url, headers=headers, data=json.dumps(request_body))
        return post_response

    def get_request(
        self,
        post_request_response
    ):
        """Requests the previously created forecasts.
        Input:
            post_request_response:request response: The output from a previous POST request that initiated the creation of a prob forecast.
        Output:
            The response from the API.
        """
        assert post_request_response.status_code == 200, f"Post request status code was not 200, status: {post_response.status_code}"
        headers = self.create_headers()
        post_json = post_request_response.json()
        request_id = post_json.get('ProbabilisticForecastRequestId', None)
        url = f"{self.pfx_url}Result/{request_id}"
        get_response = requests.request("GET", url, headers=headers, data={})
        return get_response   

    def parse_response(self, res_json):
        """Parse the response to the request asking for the prob forecasts. Derive the issue_time, lead_time from the response.
        Input:
            res_json:dict: The payload of the GET request response after it has been converted to a dict by the .json() method.
        Output:
            The forecasts in a Pandas dataframe with columns:
                "issue_time": the time from which the forecasts were created.
                "interval_start": The beginning of the period being forecasted.
                "interval_end": The end of the period being forecasted.
                "lead_time_hours": The number of decimal hours of lead time from the issue_time to the interval_start.
                "GHI": The quantiles of GHI based on the requested "percentages" in the POST request to the API.
                "POAI": The plane-of-array for a single axis tracker or fixed PV system.
        """
        status = res_json.get('Status')
        if status == 'Pending':
            print("The request is still pending")
            return None
        first_interval_start = datetime.fromisoformat(res_json.get('StartTime'))
        api_call_time = datetime.fromisoformat(res_json.get("TimeGenerated"))
        self.percentiles = res_json.get("Percentiles")
        # the time from which the forecast is created is set at at least 4 hours before the start_time.
        # Tha actual issue time is the earlier of either 1.) when the API was called, or 2.) 4 hours before the start time.
        forecast_issue_time = min(first_interval_start - timedelta(hours=4), api_call_time)
        GHI = res_json.get('Results').get('GHI', None)
        POAI = res_json.get('Results').get('POAI', None)
        num_forecasts = max([len(i) for i in [GHI, POAI] if i is not None])
        interval_starts = [first_interval_start + timedelta(minutes=i*60) for i in range(num_forecasts)]
        interval_ends = [interval_start + timedelta(minutes=60) for interval_start in interval_starts]
        lead_time_hours = [round((int_start - forecast_issue_time).total_seconds()/3600.0, 2) for int_start in interval_starts]
        forecast_dict = {
            "issue_time": forecast_issue_time,
            "interval_start": interval_starts,
            "interval_end": interval_ends,
            "lead_time_hours": lead_time_hours,
            "GHI": GHI, 
            "POAI": POAI
        }
        return pd.DataFrame(forecast_dict)

    def get_forecast(
        self,
        request_body:dict
    ):
        """
        Wrapper around asynchronous process of getting a probabilistic forecast.
        Input:
            request_body:dict: The specifications for the prob forecast request.
                See: https://apidocs.solaranywhere.com/#59e35b6b-91b7-47cc-94a7-fffbbb6f2363
        Output:
            The forecasts in a Pandas dataframe with columns:
                "issue_time": the time from which the forecasts were created.
                "interval_start": The beginning of the period being forecasted.
                "interval_end": The end of the period being forecasted.
                "lead_time_hours": The number of decimal hours of lead time from the issue_time to the interval_start.
                "GHI": The quantiles of GHI based on the requested "percentages" in the POST request to the API.
                "POAI": The plane-of-array for a single axis tracker or fixed PV system.
        """
        # start creating the forecast with a POST request.
        # store it as an instance variable if we need to use it later.
        self.post_response = self.post_request(request_body)

        time.sleep(20)
        for i in range(30):
            print(f"checking for a response at {20+i*10} seconds")
            # Get the GET request response
            self.forecast_response = pfx_handler.get_request(self.post_response)
            forecast_df = pfx_handler.parse_response(self.forecast_response.json())
            if forecast_df is not None:
                # this means the forecast is created.
                break
            elif i == 8:
                print("Final attempt")
            time.sleep(10)
        return forecast_df

    def create_forecast_interval_chart(
        self,
        forecast_df,
        vis_column = "POAI",
        figsize=(14,7)
    ):
        """Creates a chart visualizing the overlapping interquantile ranges of a forecast.
        Inputs:
            forecast_df (pd.DataFrame): The forecast dataframe output from the get_forecast method.
            vis_column (string): The column of forecast quantiles to use for visualization.
            figsize (tuple): size of the figure passed to plt.subplots.
        Output:
            fig (matplotlib figure): The figure with overlapping percentile ranges.
        """
        # import only in this method to make installing matplotlib optional.
        plt = importlib.import_module('.pyplot', package='matplotlib')

        fig, ax = plt.subplots(figsize=figsize)
        alpha = 0.15

        labels = []
        # Assumes percentiles are centered around 50.
        for i in range(len(self.percentiles) // 2):
            labels.append(f"{self.percentiles[i]}-{self.percentiles[-(i+1)]}% Interval")
        # If 50 is included, then the number of perecentiles is odd, and we plot the median.
        if len(self.percentiles) % 2 == 1:
            labels.append(f"Median ({self.percentiles[len(self.percentiles) // 2]}%)")

        for i, label in zip(range(len(labels)), labels):
            ax.fill_between(
                forecast_df['interval_start'], 
                forecast_df[vis_column].apply(lambda x: x[i] if len(x) > i else None),
                forecast_df[vis_column].apply(lambda x: x[-(i+1)] if len(x) > i else None),
                color='blue',
                alpha=alpha,
                label=label
            )
            
        ax.legend()
        ax.set_xlabel("Time")
        ax.set_ylabel(f"{vis_column} (W/m2)")
        ax.set_title(f"Forecast with Prediction Intervals, latitude {round(lat, 2)} longitude {round(lon, 2)}")
        plt.show()
        return fig
        

# Setup

In [ ]:
# Create a location
lat = 38.9364
lon = -77.2645
name = "Wolf Trap"
percentiles = [10,20,30,40,50,60,70,80,90]

#get the SolarAnywhere API key
solaranywhere_api_key = None

In [ ]:
### Create pfx_handler instance
pfx_handler = PFXHandler(api_key = solaranywhere_api_key)

# Forecast Period

## Current Day-Ahead Forecast

The "StartTime" parameter specifies the beginning of the first forecasted time interval requested. The the forecast "issue time" or "reference time", i.e. the time from which the forecast is created is internally set to 4 hours before the "StartTime". We set a start time 5 hours in the future to ensure we have the most current NWP inputs to the model. (The "issue_time" constrains the NWP inputs by only using forecasts that would have existed before that timestamp. Setting an "issue_time" in the future ensures the model can use the most recent NWP inputs.) 

In [ ]:
### specify the StartTime for the forecast such that newest NWP data is used
current_time = datetime.now()
# advance "StartTime" more than 4 hours from the current time.
forecast_start_time = current_time + timedelta(hours=5)
# reformat "StartTime" because API only accepts timestamps truncated to the hour.
forecast_start_time = forecast_start_time.replace(minute=0,second=0, microsecond=0, tzinfo=timezone.utc)
start_time_str = forecast_start_time.isoformat()
print(f"Beginning of first predicted interval: {start_time_str}")

## Past Forecasts (Backtesting)

For backtesting, set the "StartTime" to the beginning of the set of desired forecasted periods, or to four hours after when the forecast would have been issued.

In [ ]:
# Day-ahead forecast created at 10 AM EST on February 13, 2024 for Februrary 14.
EST_offset = -4
forecast_issue_time = datetime(
    year=2023,
    month=2,
    day=13,
    hour=10+EST_offset,
    tzinfo=timezone.utc
)
forecast_start_time = forecast_issue_time + timedelta(hours=4)
start_time_str = forecast_start_time.isoformat()
print(f"Forecast issued at: {forecast_issue_time.isoformat()}, beginning of first predicted interval: {start_time_str}")

# Tracking

## Single Axis Tracking POAI

We specify the pvlib singleaxistracker parameters axis_azimuth and axis_tilt for modeling the plane-of-array irradiance.
(https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.tracking.singleaxis.html)

In [ ]:
# specify single axis tracker parameters and interpret them as pfx api inputs.
axis_azimuth = 0
axis_tilt = 0
array_config = {
    "Tracking": "SingleAxis",
    "Tilt_Degrees": axis_tilt,
    "Azimuth_Degrees": axis_azimuth
}

## Fixed Mount

We specify the pvlib FixedMount parameters surface_tilt and surface_azimuth for modeling the plane-of-array irradiance.
(https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.pvsystem.FixedMount.html)

In [ ]:
# specify fixed mount parameters and interpret them as pfx api inputs.
surface_tilt = 35
surface_azimuth = 180 #south facing
array_config = {
    "Tracking": "Fixed",
    "Tilt_Degrees": surface_tilt,
    "Azimuth_Degrees": surface_azimuth
}

## POAI vs GHI

During the creation of the probabilistic forecast, all three irradiance components are modeled as a joint probability: in that the way DHI, GHI, and DNI vary is not independent.
The POAI output of the probabilistic forecast takes this into account, and the GHI output is derived from the marginal distribution of the GHI alone.
TLDR: Decomposing the GHI output to create a POAI or power forecast is likel less accurate than using the POAI output. 

# Get Forecasts from the API

## Assemble Post Request

In [ ]:
request_body = {
    "Latitude": lat,
    "Longitude": lon,
    "Name": name,
    "StartTime": start_time_str,
    "ForecastHorizon_Hours": 35,
    "Percentiles": percentiles,
    "ArrayConfiguration": array_config,
    "OutputFields": [
        "GHI",
        "POAI"
    ]
}
# View the parameters for the probabilistic forecast.
print(request_body)

## Request the Forecast

In [ ]:
forecast_df = pfx_handler.get_forecast(request_body)

In [ ]:
# take a look at the dataframe output
forecast_df.head()

## Visualize the forecast intervals

In [ ]:
fig = pfx_handler.create_forecast_interval_chart(forecast_df)